In [1]:
import bql
import pandas as pd
import datetime as datetime
from datetime import date
from collections import OrderedDict
import mibian
# Access to dedicated vizualisation libraries 
import bqviz as bqv 
# Access to open source vizualisation libraries 
import plotly.express as px

bq = bql.Service()

In [2]:
fut_ticker = 'SPX Index'
rate = 'USDR1T Curncy'

price = bq.data.px_last(fill='PREV')
current_vol = bq.data.implied_volatility(fill = 'PREV')

req_ir = bql.Request(rate, {'rate': price})
res_ir = bq.execute(req_ir)
ir_df = bql.combined_df(res_ir)
ir = float(ir_df['rate'])/100

req_fut_data = bql.Request(fut_ticker, {'fut_price': price})
res_fut_data = bq.execute(req_fut_data)
fut_df = pd.DataFrame({r.name : r.df()[r.name] for r in res_fut_data})
fut_px = int(fut_df['fut_price'])

In [3]:
def convert_to_csv(df, fut_ticker, expiry_str, moneyness_lb, moneyness_hb):
    df.reset_index()
    df = df[(df.moneyness > moneyness_lb) & (df.moneyness < moneyness_hb)]
    df_str = str(fut_ticker+''+expiry_str+'.csv')
    df.to_csv(df_str)

In [4]:
def req_option_chain(ticker, rate, fut_px, put_call, mat_list, 
                     moneyness_lb = 98, moneyness_hb = 102, front_contract = None):

        op_ticker = bq.univ.options(ticker)
        price = bq.data.px_last(fill='PREV') 
        open_interest = bq.data.open_int(fill='PREV')
        strike = bq.data.strike_px()
        implied_vol = bq.data.IVOL()
        delta = bq.data.DELTA()
        vega = bq.data.vega()
        gamma = bq.data.GAMMA()
          
        for j in range(0,len(mat_list)):
            expiry = datetime.datetime(mat_list[j][0],mat_list[j][1],mat_list[j][2])
            expiry_str = expiry.strftime("%Y-%m-%d")
            #Conditions
            put_call_cond = bq.data.put_call() == put_call 
            expiry_cond = bq.data.expire_dt() == expiry_str
            conditions = bq.func.and_(put_call_cond, expiry_cond)
            op_data_filter = bq.univ.filter(op_ticker, conditions)

            items = OrderedDict()
            items['strike'] = strike
            items['opt_price'] = price
            items['ivol'] = implied_vol
            items['open_int'] = open_interest
            items['delta'] = delta
            items['vega'] = vega
            items['gamma'] = gamma

            req = bql.Request(op_data_filter, items)
            res = bq.execute(req)
            data = pd.DataFrame({r.name:r.df()[r.name] for r in res})
            data.sort_values(by=['strike'], inplace=True)

            dte = expiry - datetime.datetime.now()

            data.insert(1, 'dte', dte.days, True)
            data.insert(2, 'dte_yrs',(data['dte']/365), True)
            data.insert(3, 'rate', rate, True)
            data.insert(4, 'fut_price', fut_px, True)
            
            call_moneyness = (fut_px / data.strike)*100
            put_moneyness = (data.strike / fut_px )*100

            if put_call == 'CALL':
                data.insert(5, 'moneyness', call_moneyness, True)
            else:
                data.insert(5, 'moneyness', put_moneyness, True)
                
            data.drop_duplicates(['strike'], keep='first', inplace=True) #Tackle BB's duplicate strikes
            
            if front_contract is None:
                data['ivol'].interpolate(inplace = True)
                data['ivol'].bfill(inplace = True)
                convert_to_csv(data, fut_ticker, expiry_str, moneyness_lb, moneyness_hb)
                data = data.set_index('strike')
                
                return data[(data.moneyness > moneyness_lb) & (data.moneyness < moneyness_hb)]
            
            else:
                data['ivol'].interpolate(inplace = True)
                data = data.set_index('strike')
                dfdif = front_contract[(front_contract.isin(data.index))]
                dfdif['ivol'] = data['ivol']
                dfdif['ivol'].interpolate(inplace = True)
                dfdif['ivol'].bfill(inplace = True)
                for i,r in dfdif.iterrows():
                    dfdif['fut_price'].at[i] = fut_px
                    dfdif['dte'].at[i] = dte.days
                    dfdif['dte_yrs'].at[i] = dte.days/365
                    dfdif['rate'].at[i] = ir
                    dfdif['open_int'].at[i] = 0
                    
                dfdif = dfdif.reset_index()
                for i,r in dfdif.iterrows():
                    if put_call == 'CALL':
                        bs = mibian.BS([r['fut_price'], r['strike'], r['rate'], r['dte']], volatility = r['ivol'])#callPrice = r['opt_price']
                        dfdif['moneyness'].at[i] = (fut_px / dfdif['strike'].at[i])*100
                        dfdif['opt_price'].at[i] = bs.callPrice
                        dfdif['delta'].at[i] = bs.callDelta
                        dfdif['vega'].at[i] = bs.vega  
                        dfdif['gamma'].at[i] = bs.gamma
                    else:
                        bs = mibian.BS([r['fut_price'], r['strike'], r['rate'], r['dte']], volatility = r['ivol'])#, putPrice = r['opt_price']
                        dfdif['moneyness'].at[i] = (dfdif['strike'].at[i] / fut_px)*100
                        dfdif['opt_price'].at[i] = bs.putPrice
                        dfdif['delta'].at[i] = bs.putDelta
                        dfdif['vega'].at[i] = bs.vega  
                        dfdif['gamma'].at[i] = bs.gamma
                        
                convert_to_csv(dfdif, fut_ticker, expiry_str, moneyness_lb, moneyness_hb)
                    #dfdif = dfdif.set_index('strike')
                #return dfdif                   

In [5]:
mat_list = [[2024,3,28]]
put_call = 'CALL'
front_contract = req_option_chain(fut_ticker, ir, fut_px, put_call, mat_list)

In [6]:
mat_list = [[2024,4,30], 
            [2024,5,31], [2024,6,28], [2024,7,31], [2024,8,16], 
            [2024,9,30], [2024,10,18], [2024,11,15], [2024,12,31]]
put_call = 'CALL'
rest_of_contracts = req_option_chain(fut_ticker, ir, fut_px, put_call, mat_list, front_contract = front_contract )